## 카메라 영상에서 얼굴 및 마스크 객체 탐지하기

In [1]:
# GPU 사용
import os
from tensorflow.compat.v1 import ConfigProto,InteractiveSession
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print(os.popen('nvidia-smi').read())

Mon Jun  3 23:46:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti   WDDM  | 00000000:01:00.0 Off |                  Off |
|  0%   44C    P2              37W / 450W |  22408MiB / 24564MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
import tensorflow as tf
from config_mask import *
from image_process import resize_to_square
from data import read_class_names
from post_process import *
from yolov3 import Create_YOLOv3
# from mqtt.CameraPublisher import ImageMqttPublisher

yolo = Create_YOLOv3(num_class=NUM_CLASS)
yolo.load_weights(os.path.join(CHECKPOINTS_FOLDER, MODEL_NAME))

weights = yolo.get_weights()
class_names = read_class_names(CLASS_LABEL_PATH)

cap = cv2.VideoCapture(0)

# 아래 코드에서 localhost는 MQTT 브로커 서버의 주소임 
# imageMqttPusblisher = ImageMqttPublisher("localhost", 1883, "/camerapub")
# imageMqttPusblisher.connect()

if cap.isOpened():
    while True:
        yolo.set_weights(weights)
        ret, image = cap.read()
        if not ret:
            print("프레임을 받지 못했습니다.")
            break 

        # 밝기를 100만큼 더함 
        dummy = np.full(image.shape, fill_value=50, 
                        dtype=np.uint8)
        cv2.add(image, dummy, image)
                
        # 콘트라스트 강조함 
        image = cv2.normalize(image, None, 0, 255,
                              cv2.NORM_MINMAX)

        # 이미지를 정사각형 모양으로 만듬 
        image_data = resize_to_square(np.copy(image), 416)
        image_data = image_data[np.newaxis,
                                ...].astype(np.float32)

        # 상자 예측 
        pred_box = yolo.predict(image_data, verbose=0)
        pred_box = [tf.reshape(x, (-1, tf.shape(x)[-1])) 
                    for x in pred_box]
        pred_box = tf.concat(pred_box, axis=0)

        # 상자 후처리 
        bboxes = postprocess_boxes(pred_box, image, 416, 0.3) # 0.3

        # NMS에 의해 해당 영역에서 상자 하나만 남김 
        bboxes = nms(bboxes, 0.45, method="nms") # 0.45

        # 상자를 그림 
        frame = draw_bbox(image, bboxes, class_names)

#         imageMqttPusblisher.sendBase64(frame)
        cv2.imshow("Image", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
else:
    print('연결된 카메라가 없습니다.')

# imageMqttPusblisher.disconnect ()
cap.release()
cv2.destroyAllWindows()